In [101]:
import pandas as pd
df = pd.read_csv('map_review.csv')
df.head()


,query,Review URL,Extracted Vendor,Number of Reviews,Average Stars,Number of 1 Star Reviews,Number of 2 Stars Reviews,Number of 3 Stars Reviews,Number of 4 Stars Reviews,Number of 5 Stars Reviews,Name,Number of Stars,Comment,Time,Owner Response,Owner Response Time,API URL
0,14 Hills,https://www.google.com/maps/place/14+Hills/@53...,14 Hills,794.0,4.3,49.0,38.0,65.0,154.0,488.0,Chlo C,4,Had a lovely time at 14 hills. The food was de...,2022-02-05 15:12:04,None,None,https://www.google.com/maps/preview/review/lis...
1,14 Hills,https://www.google.com/maps/place/14+Hills/@53...,14 Hills,794.0,4.3,49.0,38.0,65.0,154.0,488.0,Matt Coles,5,"Lovely setting, lovely food, we had lobster an...",2022-04-16 18:32:39,"Dear Mr Coles,Thank you for the wonderful revi...",2022-04-19 17:15:12,https://www.google.com/maps/preview/review/lis...
2,14 Hills,https://www.google.com/maps/place/14+Hills/@53...,14 Hills,794.0,4.3,49.0,38.0,65.0,154.0,488.0,Catherine Rickard,5,Today... went to 14 Hills for dinner with my ...,2022-01-18 23:51:24,"Dear Mrs Rickard,Thank you for your feedback!...",2022-01-18 21:30:04,https://www.google.com/maps/preview/review/lis...
3,14 Hills,https://www.google.com/maps/place/14+Hills/@53...,14 Hills,794.0,4.3,49.0,38.0,65.0,154.0,488.0,Bahar,4,This was a very pretty restaurant with good se...,2022-01-25 14:30:56,"Dear B,Thank you for finding the time to leave...",2022-01-25 19:09:28,https://www.google.com/maps/preview/review/lis...
4,14 Hills,https://www.google.com/maps/place/14+Hills/@53...,14 Hills,794.0,4.3,49.0,38.0,65.0,154.0,488.0,Kibibe Bailey,4,14 Hills is a good spot. Definitely somewhere ...,2022-02-05 19:35:02,None,None,https://www.google.com/maps/preview/review/lis...


In [102]:
df.drop_duplicates(inplace=True)
df.shape

(23990, 17)

In [103]:
df.nunique()

query                         1162
Review URL                    1173
Extracted Vendor              1143
Number of Reviews              840
Average Stars                   25
Number of 1 Star Reviews       182
Number of 2 Stars Reviews      162
Number of 3 Stars Reviews      301
Number of 4 Stars Reviews      491
Number of 5 Stars Reviews      731
Name                         19756
Number of Stars                  6
Comment                      22477
Time                         22690
Owner Response                8570
Owner Response Time           8858
API URL                       2429
dtype: int64

In [104]:
def validate_extracted(actual, extracted):
    actual_words = actual.replace('-',' ').replace(',','').replace("'", '').lower().split()
    extracted_words = extracted.replace('-',' ').replace(',','').replace("'", '').lower().split()
    match_num = 0
    for word in extracted_words:
        if word in actual_words:
            match_num += 1
        
    if match_num > len(actual_words)//2:
        return True
    else:
        return False

## Fiter to get only 5 star ratings and validate

In [106]:
import numpy as np
df['Number of Stars']= df['Number of Stars'].replace('None', np.nan)

In [107]:
df['Number of Stars'] = df['Number of Stars'].astype('float')

In [108]:
df.sort_values(by='Number of Stars', ascending=False, inplace=True)

In [109]:
five_star_dfs = []
for vendor, extracted_name in df[['query','Extracted Vendor']].drop_duplicates().values:
    vendor_df = df[df['query'] == vendor]
    five_star_df  = vendor_df[vendor_df['Number of Stars'] == 5]
    is_validated = validate_extracted(vendor,extracted_name)

    if len(five_star_df) >= 5 and is_validated:
        five_star_dfs.append(five_star_df.iloc[:5, :])
    elif len(vendor_df) >= 5 and is_validated:
        five_star_df.sort_values(by='Number of Stars', ascending=False, inplace=True)
        five_star_dfs.append(five_star_df.iloc[:5, :])
    elif is_validated:
        five_star_dfs.append(five_star_df)


clean_df = pd.concat(five_star_dfs, axis=0, ignore_index=True)

    


/tmp/ipykernel_149616/1886995713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  five_star_df.sort_values(by='Number of Stars', ascending=False, inplace=True)


In [114]:
clean_df.rename(columns={'query':'Vendor'}, inplace=True)
clean_df.nunique()

Vendor                        970
Review URL                    971
Extracted Vendor              961
Number of Reviews             739
Average Stars                  23
Number of 1 Star Reviews      166
Number of 2 Stars Reviews     147
Number of 3 Stars Reviews     280
Number of 4 Stars Reviews     442
Number of 5 Stars Reviews     658
Name                         4586
Number of Stars                 1
Comment                      4725
Time                         4781
Owner Response               1879
Owner Response Time          1934
API URL                      1562
dtype: int64

In [115]:
clean_df.to_csv('googlemap_reviews.csv', index=None)

In [78]:
fulll_df = pd.read_csv('map_review.csv')


In [112]:
vendor_df =pd.read_csv('Vendors to scrape - Sheet1.csv')
vendor_df.nunique()

Vendor Name    1231
Author         1228
Unnamed: 2        1
dtype: int64

In [80]:
scraped = fulll_df['query'].unique()

In [81]:
not_scraped = vendor_df[~vendor_df['Vendor Name'].isin(scraped)]
not_scraped

,Vendor Name,Author,Unnamed: 2
0,100 Queen’s Gate,100queensgate,NaN
2,116 Pall Mall,116pallmall,NaN
28,Amazing Grace,amazinggraceldn,NaN
31,Angel London N6,theangelhighgate,NaN
32,Angela Coomey,angelacoomey,NaN
...,...,...,...
1198,We Are FSTVL,wearefstvl,NaN
1201,West End LIVE,westendlive,NaN
1209,White Post,whitepostlondon,NaN
1217,Wimbledon,wimbledon,NaN


In [54]:
not_scraped.to_csv('filr.csv', index=None)